In [67]:
from ase.build import fcc111, molecule
from ase.visualize import view
from acat.adsorption_sites import SlabAdsorptionSites
from mace.calculators import mace_mp
from acat.build.adlayer import min_dist_coverage_pattern
from ase.optimize import BFGS
from acat.build import add_adsorbate_to_site
from ase.io import write, read
import copy
import numpy as np

In [83]:
tasks = np.arange(-6., 0., 0.5)
tasks

array([-6. , -5.5, -5. , -4.5, -4. , -3.5, -3. , -2.5, -2. , -1.5, -1. ,
       -0.5])

In [28]:
mace_calc = mace_mp()

Using Materials Project MACE for MACECalculator with /home/riccardo/.cache/mace/5yyxdm76
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.
Default dtype float32 does not match model dtype float64, converting models to float32.


In [76]:
slab = fcc111('Ag', (4, 4, 4), vacuum=5., orthogonal=True, periodic=True)
layers = {x : [] for x in np.unique(slab.positions[:,2])}

for x in slab:
    layers[x.position[2]].append(x.index)
slab = min_dist_coverage_pattern(slab, 'H', min_adsorbate_distance=2, surface='fcc111')

view(slab)

<Popen: returncode: None args: ['/home/riccardo/anaconda3/envs/npl/bin/pytho...>

In [79]:
layer_i = 0
for layer, idxs in layers.items():
    slab_copy = copy.deepcopy(slab)
    for x in idxs:
        slab_copy[x].symbol = 'Pd'
    slab_copy = relax(slab_copy)
    print(slab_copy.get_potential_energy())
    write(f'xyz/{slab_copy.get_chemical_formula()}_layer{layer_i}.xyz', slab_copy)
    layer_i += 1
    

-255.69667053222656


/home/riccardo/anaconda3/envs/npl/lib/python3.11/site-packages/ase/io/extxyz.py:302: UserWarning: Skipping unhashable information adsorbate_info


-260.9299621582031
-258.9673156738281
-268.3700866699219


In [45]:
slab = fcc111('Ag', (4, 4, 4), vacuum=5., orthogonal=True, periodic=True)
slab[58].symbol = 'Pd'
slab = relax(slab)
print(slab.get_potential_energy())

-173.6931610107422


In [16]:
slab = fcc111('Ag', (4, 4, 4), vacuum=5., orthogonal=True, periodic=True)
slab[22].symbol = 'Pd'
slab = relax(slab)
print(slab.get_potential_energy())

-173.86782836914062


<Popen: returncode: None args: ['/home/riccardo/anaconda3/envs/npl/bin/pytho...>

In [42]:
sas = SlabAdsorptionSites(slab, surface='fcc111',
                          composition_effect=True)

surface_ads = [] 
sampled = []
for site in sas.get_sites():
    if 'Pd' in site['composition'] and site['site'] not in sampled:
        slab_copy = copy.deepcopy(slab)
        add_adsorbate_to_site(slab_copy, adsorbate='H', site=site)
        surface_ads.append((slab_copy, site['site']))
        sampled.append(site['site'])

In [49]:
h2 = relax(molecule('H2'))

In [51]:
for atoms, site in surface_ads:
    relax(atoms)
    print(atoms.get_potential_energy() - (slab.get_potential_energy() + h2.get_potential_energy()/2), site)

-0.30016326904296875 fcc
-0.29282379150390625 hcp
-0.24587249755859375 bridge
0.07137298583984375 ontop


In [13]:
def relax(atoms, single_point=False):
    atoms.calc = mace_calc
    if not single_point:
        opt = BFGS(atoms, logfile=None)
        opt.run(fmax=0.03)
    return atoms

<Popen: returncode: None args: ['/home/riccardo/anaconda3/envs/npl/bin/pytho...>

In [52]:
view(slab)

<Popen: returncode: None args: ['/home/riccardo/anaconda3/envs/npl/bin/pytho...>